### Summary of Notebook
- In this notebook , we will pre-process the input data to make it compatible for model building
- First do a Train-Test split and create 2 subsets (70:30 split)
- We will convert all images to tensors for both subsets
- Resize all the images to one pre-defined size, convert bounding boxes accordingly
- Visualize data and see that everything looks good
- At the end of this program , data will be  ready for model building

In [ ]:
# Import Genereal libs  libraries

import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import sys
import numpy as np
import seaborn as sns
import statistics as stats
sns.set(color_codes=True)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#Last import allows multiple outputs from one cell
import warnings
# Initialize the random number generator
import pickle
import random
random.seed(101)
# Import project specific libs
import tarfile
import os
import matplotlib.image as img
import cv2
import matplotlib.patches as patches
import tensorflow as tf
import pickle
from sklearn.model_selection import train_test_split


In [ ]:
print(tf.__version__)

2.9.2


In [ ]:
# Useful Configuration/Setting

# suppress display of warnings
warnings.filterwarnings('ignore')

# display all dataframe columns
pd.options.display.max_columns = None

# to set the limit to 3 decimals
pd.options.display.float_format = '{:.7f}'.format

# display all dataframe rows
pd.options.display.max_rows = None

#Setting to shows all entries in array displayed
np.set_printoptions(threshold=sys.maxsize)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
class Preprocess_Util:
  #This Utility class performs EDA on the input data

  #Constructor func
  def __init__(self):
    self.imageFilepath="/content/drive/MyDrive/Object_Localisation/Data_Set/images/"
    self.ip_dataframe=pickle.load(open('/content/drive/MyDrive/Object_Localisation/Pickled_Data/IpDataFrame.pickle','rb'))
    print("# Shape #")
    # Add a filename extension
    self.ip_dataframe['filename']=self.ip_dataframe['image_name'] +'.jpg'
    print(self.ip_dataframe.shape)
    print("##########################################################################")
    #print("# Sample Records #")
    #print(self.ip_dataframe.sample(5))
    #print("##########################################################################")
    #print("# First 5  Records #")
    #print(self.ip_dataframe.head(5))
    #print("##########################################################################")
    self.rowSize=self.ip_dataframe.shape[0]
    #image length and width we desire
    self.expImageLength=256
    self.expImageWidth=256
    #Check if pickled Train tensors already exist as part of the previously executed run of the program
    if(os.path.isfile('/content/drive/MyDrive/Object_Localisation/Pickled_Data/X_Train.npy')):
      self.flagFormatTrainData=False
    else:
      self.flagFormatTrainData=True
    #Check if pickled Test tensors already exist as part of the previously executed run of the program
    if(os.path.isfile('/content/drive/MyDrive/Object_Localisation/Pickled_Data/X_Test.npy')):
      self.flagFormatTestData=False
    else:
      self.flagFormatTestData=True
  
  def splitTrainTest(self):
    self.train_df,self.test_df=train_test_split(self.ip_dataframe,test_size=.30,random_state=0,stratify=self.ip_dataframe['id'],shuffle=True)
    # Check stats on splits
    print("train_df.shape ",self.train_df.shape)
    print("##########################################################################")
    print("test_df.shape ",self.test_df.shape)
    print("##########################################################################")
    #print("Check label balance in Train ",(self.train_df['id'].value_counts(normalize=True)).sort_index())
    #print("##########################################################################")
    #print("Check label balance in Test ", (self.test_df['id'].value_counts(normalize=True)).sort_index())
    #print("##########################################################################")
    #Lets check if the records were shuffled well
    #print("First 5 sample records of Train",self.train_df.head(100))
    #print("##########################################################################")
    #print("First 5 sample records of Test ",self.test_df.head(5))
    #print("##########################################################################")
    return


  def formatTrainData(self):
    if(self.flagFormatTrainData == True):
      # Convert Data to an appropraite form for modelling
      # Since we are going to build the model from scrach using tensor flow, we will convert data to tensors
      # create Image tensor
      X_Train=np.zeros([self.train_df.shape[0],self.expImageLength,self.expImageLength,3])
      # Regression Labels
      Y_Train_RegLabel=np.zeros([self.train_df.shape[0],4])
      # Classification labels
      Y_Train_ClassLabel=np.zeros([self.train_df.shape[0],1])
      # Loop over dataframe and create tensors
      for counter in np.arange(0,self.train_df.shape[0],1) :
        eachRecord=self.ip_dataframe.iloc[counter]
        #Get path
        eachImagePath=self.imageFilepath + str(eachRecord['filename'])
        eachIm=img.imread(eachImagePath)
        #Get Size
        eachImLength=eachIm.shape[0]
        eachImWidth=eachIm.shape[1]
        # Resize Images
        eachImgResized=cv2.resize(eachIm, dsize=(self.expImageLength, self.expImageWidth), interpolation=cv2.INTER_CUBIC)
        #Get BB box
        eachImgXmin=eachRecord['xmin']
        eachImgYmin=eachRecord['ymin']
        eachImgXmax=eachRecord['xmax']
        eachImgYmax=eachRecord['ymax']
        #Resize BB
        eachImgXmin=int(int(eachImgXmin)*self.expImageWidth/eachImWidth)
        eachImgXmax=int(int(eachImgXmax)*self.expImageWidth/eachImWidth)
        eachImgYmin=int(int(eachImgYmin)*self.expImageLength/eachImLength)
        eachImgYmax=int(int(eachImgYmax)*self.expImageLength/eachImLength)
        # Append tensors
        X_Train[counter]=eachImgResized
        Y_Train_RegLabel[counter]=[eachImgXmin,eachImgYmin,eachImgXmax,eachImgYmax]
        #NTS optionally can also do pd.Series(data=[eachImgXmin,eachImgYmin,eachImgXmax,eachImgYmax])
        Y_Train_ClassLabel[counter]=eachRecord['id']

      #Lets check sizes of our tensors
      print("# Tensor Data #")
      print('counter :',counter)
      print('X Image Tensor Size: ',X_Train.shape)
      print('Y Reg  Label Size: ',Y_Train_RegLabel.shape)
      print('Y Class Label Size: ',Y_Train_ClassLabel.shape)
      print("##########################################################################")
      # pickle Tensor Data
      np.save('/content/drive/MyDrive/Object_Localisation/Pickled_Data/X_Train.npy',X_Train)
      np.save('/content/drive/MyDrive/Object_Localisation/Pickled_Data/Y_Train_RegLabel.npy',Y_Train_RegLabel)
      np.save('/content/drive/MyDrive/Object_Localisation/Pickled_Data/Y_Train_ClassLabel.npy',Y_Train_ClassLabel)
      #NTS: Pickle.dump func available on certain objects like df, for tensors , use np.save which is the same thing

    return


  def formatTestData(self):
    if(self.flagFormatTestData == True):
      # Convert Data to an appropraite form for modelling
      # Since we are going to build the model from scrach using tensor flow, we will convert data to tensors
      # create Image tensor
      X_Test=np.zeros([self.test_df.shape[0],self.expImageLength,self.expImageLength,3])
      # Regression Labels
      Y_Test_RegLabel=np.zeros([self.test_df.shape[0],4])
      # Classification labels
      Y_Test_ClassLabel=np.zeros([self.test_df.shape[0],1])
      # Loop over dataframe and create tensors
      for counter in np.arange(0,self.test_df.shape[0],1) :
        eachRecord=self.ip_dataframe.iloc[counter]
        #Get path
        eachImagePath=self.imageFilepath + str(eachRecord['filename'])
        eachIm=img.imread(eachImagePath)
        #Get Size
        eachImLength=eachIm.shape[0]
        eachImWidth=eachIm.shape[1]
        # Resize Images
        eachImgResized=cv2.resize(eachIm, dsize=(self.expImageLength, self.expImageWidth), interpolation=cv2.INTER_CUBIC)
        #Get BB box
        eachImgXmin=eachRecord['xmin']
        eachImgYmin=eachRecord['ymin']
        eachImgXmax=eachRecord['xmax']
        eachImgYmax=eachRecord['ymax']
        #Resize BB
        eachImgXmin=int(int(eachImgXmin)*self.expImageWidth/eachImWidth)
        eachImgXmax=int(int(eachImgXmax)*self.expImageWidth/eachImWidth)
        eachImgYmin=int(int(eachImgYmin)*self.expImageLength/eachImLength)
        eachImgYmax=int(int(eachImgYmax)*self.expImageLength/eachImLength)
        # Append tensors
        X_Test[counter]=eachImgResized
        Y_Test_RegLabel[counter]=[eachImgXmin,eachImgYmin,eachImgXmax,eachImgYmax]
        #NTS optionally can also do pd.Series(data=[eachImgXmin,eachImgYmin,eachImgXmax,eachImgYmax])
        Y_Test_ClassLabel[counter]=eachRecord['id']

      #Lets check sizes of our tensors
      print("# Tensor Data #")
      print('counter :',counter)
      print('X Image Tensor Size: ',X_Test.shape)
      print('Y Reg  Label Size: ',Y_Test_RegLabel.shape)
      print('Y Class Label Size: ',Y_Test_ClassLabel.shape)
      print("##########################################################################")
      # pickle Tensor Data
      np.save('/content/drive/MyDrive/Object_Localisation/Pickled_Data/X_Test.npy',X_Test)
      np.save('/content/drive/MyDrive/Object_Localisation/Pickled_Data/Y_Test_RegLabel.npy',Y_Test_RegLabel)
      np.save('/content/drive/MyDrive/Object_Localisation/Pickled_Data/Y_Test_ClassLabel.npy',Y_Test_ClassLabel)
      #NTS: Pickle.dump func available on certain objects like df, for tensors , use np.save which is the same thing

    return

  def visualizeTrainImages(self,noOfImages):
      #Load pickled data
      self.X_Train=np.load('/content/drive/MyDrive/Object_Localisation/Pickled_Data/X_Train.npy')
      self.Y_Train_RegLabel=np.load('/content/drive/MyDrive/Object_Localisation/Pickled_Data/Y_Train_RegLabel.npy')
      self.Y_Train_ClassLabel=np.load('/content/drive/MyDrive/Object_Localisation/Pickled_Data/Y_Train_ClassLabel.npy')
      
      #Choose #noOfImages images Randomly & Display image with bounding box and label
      fig = plt.figure(figsize=(15,15))
      for i in np.arange(0,4,1):
        random.seed(i)
        genRandNum= random.randint(0,self.train_df.shape[0])
        image=self.X_Train[genRandNum]
        #print("##########################################################################")
        xmin=self.Y_Train_RegLabel[genRandNum][0]
        xmax=self.Y_Train_RegLabel[genRandNum][2]
        ymin=self.Y_Train_RegLabel[genRandNum][1]
        ymax=self.Y_Train_RegLabel[genRandNum][3]
        label=self.Y_Train_ClassLabel[genRandNum][0]

        # Display Image with bb
        ax=plt.subplot(noOfImages/2,2,i+1)
        plt.grid(False)
        plt.imshow(image/255)
        rect = patches.Rectangle((int(xmin), int(ymin)), int(xmax) - int(xmin), int(ymax) - int(ymin), linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        plt.title(label)
      return

  def visualizeTestImages(self,noOfImages):
      #Load pickled data
      self.X_Test=np.load('/content/drive/MyDrive/Object_Localisation/Pickled_Data/X_Test.npy')
      self.Y_Test_RegLabel=np.load('/content/drive/MyDrive/Object_Localisation/Pickled_Data/Y_Test_RegLabel.npy')
      self.Y_Test_ClassLabel=np.load('/content/drive/MyDrive/Object_Localisation/Pickled_Data/Y_Test_ClassLabel.npy')
      
      #Choose #noOfImages images Randomly & Display image with bounding box and label
      fig = plt.figure(figsize=(15,15))
      for i in np.arange(0,4,1):
        random.seed(i)
        genRandNum= random.randint(0,self.test_df.shape[0])
        image=self.X_Test[genRandNum]
        #print("##########################################################################")
        xmin=self.Y_Test_RegLabel[genRandNum][0]
        xmax=self.Y_Test_RegLabel[genRandNum][2]
        ymin=self.Y_Test_RegLabel[genRandNum][1]
        ymax=self.Y_Test_RegLabel[genRandNum][3]
        label=self.Y_Test_ClassLabel[genRandNum][0]

        # Display Image with bb
        ax=plt.subplot(noOfImages/2,2,i+1)
        plt.grid(False)
        plt.imshow(image/255)
        rect = patches.Rectangle((int(xmin), int(ymin)), int(xmax) - int(xmin), int(ymax) - int(ymin), linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        plt.title(label)
      return


In [ ]:
if __name__ == "__main__":
  objPreproc= Preprocess_Util()
  objPreproc.splitTrainTest()
  objPreproc.formatTrainData()
  objPreproc.formatTestData()
  objPreproc.visualizeTrainImages(4)
  objPreproc.visualizeTestImages(4)

Output hidden; open in https://colab.research.google.com to view.

### Inferences
- Data has been preprocessed into tensors and looks ready for model building
